# Demande immobilière en France

Par Yanis Larré, Tom Croquette et Renan Romariz.


## Introduction 

Nous nous proposons ici d'étudier la demande immobilière en France. Mettre de la biblio + idées + NB


## Sommaire

* [Installation](#installation)


## I. Installation
La première étape du projet consiste à télécharger tous les modules et fonctions dont nous auront besoin.

In [12]:
!pip install -r requirements.txt

# Modules :
import os
import s3fs
import pandas as pd

# Fonctions :
from scripts import get_data
from scripts import data_clean
from scripts import data_analysis
from scripts import data_visualization
from scripts import do_ols

## II. Préparation des données

Nous téléchargeons dans cette parties les données utiles dans notre projet. Nous procédons également au nettoyage de celles-ci afin de pouvoir les exploiter convenablement.

### 1. Données immobilières

Nous décidons de récupérer des données foncières sur le site *data.gouv*, en utilisant plus particulièrement les données DVF (Demande de Valeur Foncière).
Ces données concernent la quasi-totalité de la France métropolitaine et des département d'Outre-Mer. En effet, les données de Mayotte ne sont pas incluses, ainsi que celles de 3 départements métropolitains : le Bas-Rhin (67), le Haut-Rhin (68) et la Moselle (57), car ces département utilisent un système hérité de l'annexion allemande appelé le Livre Foncier. Leurs données ne sont pas centralisées par la DGFiP de la même manière, donc elles ne redescendent pas dans la base nationale DVF.


In [2]:
df = get_data.get_dvf()

/home/onyxia/work/Python_2A/scripts/get_data.py:23: DtypeWarning: Columns (12,14,16,17,18,20,22,24,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, dtype={'code_commune': "str"})


#### Nettoyage

Dans notre analyse, nous avons pris le parti de ne considérer que les maisons et appartements. En effet, la base de données contient d'autres catégories de biens immobiliers telles que les terrains vierges par exemple. Toutefois, les maisons et appartements sont les biens susceptibles de montrer les résultats les plus parlants ; ce sont pas ailleurs les catégories proposant les données le plus complètes sur le prix, la localisation et la surface notamment.

In [3]:
df = df[df['type_local'].isin(['Maison', 'Appartement'])]
df = df.reset_index()

Ensuite, les codes communes n'ont pas tous le même type : certains sont de type *int*, d'autres de type *string*. On convertit l'ensemble au type *string* car certains codes communes (notamment ceux des communes corses, commençant par 2A ou 2B) ne peuvent pas être convertis en type *int*.

In [4]:
data_clean.convertir_codes_communes(df)
print(df['code_commune'])

0          01130
1          01451
2          01364
3          01053
4          01177
           ...  
6247210    75111
6247211    75112
6247212    75116
6247213    75113
6247214    75105
Name: code_commune, Length: 6247215, dtype: object


#### Aperçus des données

En calculant le nombre de ventes par commune, on remarque que Toulouse est la commune comptant le plus de ventes entre 2020 et 2025 (50684 ventes).

In [5]:
ventes_par_commune = df['code_commune'].value_counts().reset_index(name='nombre')

# Premiers résultats :
print(ventes_par_commune)
print( ventes_par_commune["nombre"][ventes_par_commune["code_commune"]=='2A004'])
print(ventes_par_commune['nombre'].max())
print(ventes_par_commune['nombre'].idxmax())
print(ventes_par_commune['code_commune'][ventes_par_commune['nombre'].idxmax()])
print(ventes_par_commune['nombre'][ventes_par_commune['nombre'].idxmax()])

      code_commune  nombre
0            31555   50684
1            06088   46697
2            44109   30331
3            33063   30077
4            34172   28927
...            ...     ...
33265        52549       1
33266        21136       1
33267        21653       1
33268        52053       1
33269        52505       1

[33270 rows x 2 columns]
145    4880
Name: nombre, dtype: int64
50684
0
31555
50684


### 2. Données de population

Afin d'étudier la dynamique autour du secteur immobilier (restreint aux maisons et aux appartements), il nous est nécessaire d'utiliser les données de population du territoire français. Pour cela, nous utilisons les données de XXX.

In [6]:
df_pop = get_data.get_pop()

Certaines valeurs ne sont toutefois pas renseignées ; on remarque aussi que certaines communes ont des populations très faibles.
Pour que le ratio des ventes sur la population ait du sens, il nous faut éliminer les communes ayant une population très faible (en particulier, la commune de Bezonvaux ne comptant aucun habitant...).
On imposte donc un nombre minimum d'habitants arbitraire, et on exclut les communes de population trop faible ou non renseignée.

PARTIE INUTILE MAISEFFECTUE PLUS TARD ASKIP

In [7]:
lignes_na = data_clean.filtre_donnes_pop(df_pop)
print(lignes_na)

       objectid  reg dep    cv code_commune                  libgeo  p13_pop  \
3435     128717   26  21  2115        21507               Premières    140.0   
8276     116240   24  45  4504        45287    Saint-Loup-de-Gonois     97.0   
11971    143352   52  53  5306        53274                 Vimarcé    236.0   
12193    143465   54  16  1607        16351            Saint-Simeux    589.0   
19186    134748   52  53  5306        53239  Saint-Martin-de-Connée    411.0   
26133    122832   23  27  2712        27676                Venables    794.0   
28348    125987   26  21  2107        21213               Crimolois    759.0   

       p14_pop  p15_pop  p16_pop  p17_pop  p18_pop  p19_pop  p20_pop  p21_pop  
3435     140.0    141.0    141.0      141      NaN      NaN      NaN      NaN  
8276      91.0     89.0     86.0       97      NaN      NaN      NaN      NaN  
11971    242.0    241.0    240.0      237    234.0      NaN      NaN      NaN  
12193    598.0    607.0    610.0      5

Nous remarquons que certains codes communes sont à six chiffres. En effet, dans les DOMs, le numéro commence par un code département à trois chiffres plutôt qu'à deux (pour différencier les DOMs).
Cela n'est pas un problème en soit, mais la convention adoptée dans le fichier DVF est différente ; on ne rajoute pas le troisième chiffre.
Par exemple, le code commune est différent dans *df_pop* et *df* pour la commune de Salazie, à la Réunion (974421 dans *df_pop*, 97421 dans *df*) : la Réunion est perçue comme le département 974 dans *df_pop*. On va ainsi supprimer le troisième chiffre de chacun des codes communes à 6 chiffres.


In [13]:
# Problème des codes communes à 6 chiffres :
print(df_pop[df_pop['code_commune']=='974421'])
lignes_6_chiffres = df_pop[df_pop["code_commune"].str.len() == 6]
print(lignes_6_chiffres)
print(df_pop[df_pop['code_commune']=='974421']['libgeo'])
print(df[df['code_commune']=='97421']['nom_commune'])

# Harmonisation :
df_pop["code_commune"] = df_pop["code_commune"].apply(data_clean.enleverchiffreDOMs)
print(df_pop)
lignes_6_chiffres = df_pop[df_pop["code_commune"].str.len() == 6]
print(lignes_6_chiffres)
print(df_pop[df_pop['code_commune']=='974421'])

      objectid  reg  dep     cv code_commune   libgeo  p13_pop  p14_pop  \
3786    131030    4  974  97406       974421  Salazie   7226.0   7132.0   

      p15_pop  p16_pop  p17_pop  p18_pop  p19_pop  p20_pop  p21_pop  
3786   7384.0   7400.0     7312   7224.0   7136.0   7310.0   7243.0  
       objectid  reg  dep     cv code_commune                  libgeo  \
84       126394    3  973    973       973353             Maripasoula   
192      126587    3  973    973       973313  Montsinéry-Tonnegrande   
206      126601    3  973    973       973358              Saint-Élie   
210      126605    1  971  97114       971102           Anse-Bertrand   
748      130678    1  971  97195       971106              Bouillante   
...         ...  ...  ...    ...          ...                     ...   
33141    115576    2  972    972       972225            Saint-Pierre   
33257    111336    4  974  97401       974404           L' Étang-Salé   
33557    111383    2  972    972       972201      L

AttributeError: module 'scripts.data_clean' has no attribute 'enleverchiffreDOMs'